In [1]:

!pip install -qU accelerate peft bitsandbytes transformers trl huggingface_hub scipy tensorboard datasets evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 354.7/354.7 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.0/411.0 kB 4.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 23.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 108.3 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 336.4/336.4 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 481.4/481.4 kB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.6/37.6 MB 47.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 98.2 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 5.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the

In [2]:
from huggingface_hub import notebook_login

notebook_login()
print("Login successful!")

Login successful!


In [3]:
import torch
import torch.nn as nn
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from peft import LoraConfig, get_peft_model, TaskType
import evaluate

In [4]:
from transformers import BitsAndBytesConfig
bnb_config=BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [5]:
from transformers import AutoTokenizer, AutoModelForCausalLM

teacher_tokenizer = AutoTokenizer.from_pretrained("NousResearch/Meta-Llama-3-8B")
teacher_model = AutoModelForSequenceClassification.from_pretrained("NousResearch/Meta-Llama-3-8B",quantization_config=bnb_config,num_labels=2)

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/73.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/654 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at NousResearch/Meta-Llama-3-8B and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
from transformers import AutoTokenizer, AutoModelForCausalLM

student_tokenizer = AutoTokenizer.from_pretrained("NousResearch/Llama-3.2-1B")
student_model = AutoModelForSequenceClassification.from_pretrained("NousResearch/Llama-3.2-1B",num_labels=2)

tokenizer_config.json:   0%|          | 0.00/50.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at NousResearch/Llama-3.2-1B and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
student_tokenizer.pad_token = student_tokenizer.eos_token

In [8]:
from datasets import load_dataset

ds = load_dataset("stanfordnlp/sst2")

README.md:   0%|          | 0.00/5.27k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/3.11M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/72.8k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/148k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

In [ ]:
ds


In [9]:
teacher_tokenizer.pad_token = teacher_tokenizer.eos_token
def preprocess_function(examples):
    return teacher_tokenizer(examples["sentence"],padding="max_length",truncation=True,max_length=128)

In [10]:
t_ds=ds.map(preprocess_function,batched=True)

Map:   0%|          | 0/67349 [00:00<?, ? examples/s]

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

Map:   0%|          | 0/1821 [00:00<?, ? examples/s]

In [11]:
t_ds.set_format("torch", columns=["input_ids", "attention_mask", "label"])

In [12]:
t_ds

DatasetDict({
    train: Dataset({
        features: ['idx', 'sentence', 'label', 'input_ids', 'attention_mask'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['idx', 'sentence', 'label', 'input_ids', 'attention_mask'],
        num_rows: 872
    })
    test: Dataset({
        features: ['idx', 'sentence', 'label', 'input_ids', 'attention_mask'],
        num_rows: 1821
    })
})

In [13]:
teacher_model.config.pad_token_id = teacher_tokenizer.eos_token_id
student_model.config.pad_token_id = student_tokenizer.eos_token_id


In [14]:
teacher_model.eval()
for param in teacher_model.parameters():
    param.requires_grad = False

In [15]:
lora_config=LoraConfig(
    task_type="SEQ_CLS",
    r=4,
    lora_alpha=8,
    lora_dropout=0.1
)

In [16]:
student_model = get_peft_model(student_model, lora_config)

In [17]:
class NEOTrainer(Trainer):
    def __init__(self,*args,teacher_model=None,temprature=2.0,alpha=0.5,**kwargs):
        super().__init__(*args,**kwargs)
        if teacher_model is None:
            raise ValueError("A teacher model must be provided for distillation.")
        
                        
        self.teacher_model=teacher_model
        self.temprature=temprature
        self.alpha=alpha
        self.kl_loss_fn=nn.KLDivLoss(reduction="batchmean")
        self.ce_loss_fn=nn.CrossEntropyLoss()
    def compute_loss(self,model,inputs,return_outputs=False,**kwargs):
        outputs=model(**inputs)
        student_logits=outputs.logits
        labels=inputs["labels"]
        loss_ce=self.ce_loss_fn(student_logits,labels)
        with torch.no_grad():
            teacher_outputs=self.teacher_model(**inputs)
        teacher_logits=teacher_outputs.logits
        T=self.temprature
        student_log_prob=torch.log_softmax(student_logits/T,dim=-1)
        teacher_prob=torch.softmax(teacher_logits/T,dim=-1)
        loss_kd=self.kl_loss_fn(student_log_prob,teacher_prob)*(T**2)
        loss=self.alpha*loss_ce+(1-self.alpha)*loss_kd
        return (loss, outputs) if return_outputs else loss
            

In [18]:
training_args = TrainingArguments(
    output_dir="distilled-llama-sst2",
    num_train_epochs=1,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    learning_rate=5e-5,
    eval_steps=1000,
    save_steps=1000,
    bf16=True,
    eval_strategy="steps",
    logging_strategy="steps",
    save_strategy="steps",
    weight_decay=0.01,
    push_to_hub=False,
)

In [19]:
clf_metrics = evaluate.combine(["accuracy", "f1", "precision", "recall"])
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=-1)
    return clf_metrics.compute(predictions=predictions, references=labels)


In [20]:
trainer=NEOTrainer(
    model=student_model,
    teacher_model=teacher_model,
    args=training_args,
    train_dataset=t_ds["train"],
    eval_dataset=t_ds["validation"],
    temprature=2.0,
    alpha=0.5,
    compute_metrics=compute_metrics
)

No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [21]:
import wandb
wandb.login(key="2516632c2e1b8d6effd4ae16da55f5b1ff7f50a2")
wandb.init()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: ankurmishrabxr (ankurmishrabxr-rgipt-jais). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [22]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1000,0.385600,0.368122,0.924312,0.927313,0.907328,0.948198
2000,0.368100,0.363355,0.926606,0.929670,0.907725,0.952703
3000,0.364800,0.359891,0.934633,0.936596,0.925275,0.948198
4000,0.366200,0.358008,0.927752,0.930997,0.906183,0.957207


TrainOutput(global_step=4210, training_loss=0.3784728895173786, metrics={'train_runtime': 29732.6939, 'train_samples_per_second': 2.265, 'train_steps_per_second': 0.142, 'total_flos': 5.035749797108122e+16, 'train_loss': 0.3784728895173786, 'epoch': 1.0})

In [23]:
trainer.save_model()

In [24]:
student_model.cpu()
torch.cuda.empty_cache()

trainer.push_to_hub() 

training_args.bin:   0%|          | 0.00/5.30k [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/1.73M [00:00<?, ?B/s]

events.out.tfevents.1744194444.92cdd7270589.31.0:   0%|          | 0.00/9.15k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Ank110/distilled-llama-sst2/commit/409c230e1c7dbf652e0035a8eeaae9d236f9cd91', commit_message='End of training', commit_description='', oid='409c230e1c7dbf652e0035a8eeaae9d236f9cd91', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Ank110/distilled-llama-sst2', endpoint='https://huggingface.co', repo_type='model', repo_id='Ank110/distilled-llama-sst2'), pr_revision=None, pr_num=None)